# CGSN Metadata Communications
Author: Andrew Reed

Date: 2019-08-21

Ver: 1.02

This notebook lays out the development process for querying the relevant data fields necessary for CGSN to fill out the UW Metadata Changes & Communications spreadsheet. The goal is to explore the possible sources of the necessary information by M2M calls to OOINet based upon the available information recorded in CGSN's Metadata Tracking Spreadsheet. Eventually, the exploratory and development process laid out below will be transitioned into an automated function which fills out the requisite information with manual execution of the relevant scripts and code.


In [1]:
import os, shutil, sys, time, re, requests, csv, datetime, pytz
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr

/home/andrew/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


Set my OOINet username, token, and the base url for querying the system via M2M:

In [2]:
username = 'OOIAPI-C9OSZAQABG1H3U'
token = 'JA48WUQVG7F'

In [3]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m'
sensor_url = '12576/sensor/inv'
asset_url = '12587/asset'

The goal in this code is to, for a given asset. Let's start with the SPKIRs because of the small number of different individual instruments:

In [4]:
# Specify some functions to convert timestamps
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

def ntp_seconds_to_datetime(ntp_seconds):
    return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)
  
def convert_ooi_time(ms):
    if ms is None:
        return None
    elif np.isnan(ms):
        return None
    else:
        return datetime.datetime.utcfromtimestamp(ms/1000)

In [5]:
def get_cal_data(sn):
    """
    Returns calibration data for an instrument with given serial number
    """
    cal_url = '/'.join((base_url, asset_url))+'?serialnumber='+str(sn)
    cal_data = requests.get(cal_url, auth=(username, token)).json()
    return cal_data


In [6]:
def get_calData(uid):
    
    cal_url = '/'.join((base_url, '12587/asset/cal?uid=')) + uid
    calData = requests.get(cal_url, auth=(username, token)).json()
    return calData

In [7]:
def get_deployData(uid):
    """
    Function to call the m2m interface and get the deployment
    data for a given instrument uid
    """
    deploy_url = '/'.join((base_url, '12587/asset/deployments', uid)) + '?editphase=ALL'
    deployData = requests.get(deploy_url, auth=(username, token)).json()
    return deployData

In [8]:
def reformat_deployData(deployData):
    """
    Reformats the deployment information for a given instrument
    into a dataframe with the following fields: subsite, node,
    deployment number, start time, and end time.
    """
    df = pd.DataFrame(columns=['subsite','node','deploymentNumber','startTime','endTime'])
    for d in deployData:
        df = df.append({
            'subsite':d['subsite'],
            'node':d['node'],
            'deploymentNumber':d['deploymentNumber'],
            'startTime':d['startTime'],
            'endTime':d['endTime'],
        }, ignore_index=True)
        
    df['startTime'] = df['startTime'].apply(convert_time)
    df['endTime'] = df['endTime'].apply(convert_time)
    df.sort_values(by=['deploymentNumber'], inplace=True)
    
    return df

In [9]:
def reformat_calData(calData):
    caldict = {}
    for coeff in calData['calibration']:
        name = coeff['name']
        cdata = coeff['calData']
        eventStart = []
        eventStop = []
        eventMod = []
        dataSource = []
        for event in cdata:
            eventStart.append(convert_time(event['eventStartTime']))
            eventMod.append(convert_time(event['lastModifiedTimestamp']))
            dataSource.append(event['dataSource'])
        caldict.update({name:{'eventStartTime':eventStart, 'lastModifiedTimestamp':eventMod, 'dataSource':dataSource}})
    dfCal = pd.DataFrame(caldict).transpose()
    
    return dfCal

In [10]:
os.listdir(os.getcwd())

['CGSN Metadata Review.xlsx',
 'Asset_Management_Instrument_Metadata.ipynb',
 'Stream_identifier.ipynb',
 '.ipynb_checkpoints']

In [11]:
metadata_review = pd.read_excel('CGSN Metadata Review.xlsx',sheet_name='Cal Review Log')

In [12]:
metadata_review

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,Pull request #,Date pull request submitted,Pull request verified primary,Pull request verified secondary,Unnamed: 14
0,PCO2W-B,C0050,2012-08-28 00:00:00,CGINS-PCO2WB-C0050__20150315,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
1,PCO2W-B,C0050,2016-05-20 00:00:00,CGINS-PCO2WB-C0050__20161125,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
2,PCO2W-B,C0051,2012-08-28 00:00:00,CGINS-PCO2WB-C0051__20140910,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
3,PCO2W-B,C0051,2015-12-10 00:00:00,CGINS-PCO2WB-C0051__20160513,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
4,PCO2W-B,C0051,2016-12-21 00:00:00,CGINS-PCO2WB-C0051__20161221,Yes,Yes,Yes,"SW, AP",No,NaN,NaN,NaN,NaN,NaN,NaN
5,PCO2W-B,C0051,2017-11-14 00:00:00,CGINS-PCO2WB-C0051__20171114,Yes,Yes,Yes,"SW, AP",No,NaN,NaN,NaN,NaN,NaN,NaN
6,PCO2W-B,C0069,2013-08-02 00:00:00,CGINS-PCO2WB-C0069__20131121,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
7,PCO2W-B,C0069,2015-04-28 00:00:00,CGINS-PCO2WB-C0069__20151021,Yes,Yes,"No, need to change date","SW, AP",No,changed date (RGT),659,2019-03-28 00:00:00,CD,AR,NaN
8,PCO2W-B,C0069,2016-06-10 00:00:00,CGINS-PCO2WB-C0069__20160610,Yes,Yes,Yes,"SW, AP",No,NaN,NaN,NaN,NaN,NaN,NaN
9,PCO2W-B,C0069,2017-07-19 00:00:00,CGINS-PCO2WB-C0069__20170719,Yes,Yes,Yes,"SW, AP",No,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
metadata_review.dropna(subset=['CLASS-SERIES'], inplace=True)

In [14]:
metadata_review = metadata_review[metadata_review['Original Calibration CSV'] != 'Bad']

In [15]:
def generate_uid(inst, sn, whoi_inst=True):
    """
    Function which takes in instrument class - series and serial number to generate an instrument uid. The exception
    to the rule is the METBK instruments, which are classified as Loggers, and thus are recorded as METLGR
    """
    
    # Clean the names of the class-series
    if '-' in inst:
        inst = inst.replace('-','')
        
    # Clean the serial numbers
    sn = str(sn)
    if '-' in sn:
        ind = sn.index('-')
        sn = sn[ind+1:].zfill(5)
    elif len(sn) < 5:
        sn = sn.zfill(5)
    else:
        pass
    
    # If the instrument is a METBK, have to handle differently
    if 'METBKA' in inst:
        inst = 'METLGR'
        if 'UNKNOWN' in sn:
            sn = sn.split('\n')[-1]
        else:
            sn = sn[3:].zfill(5)   
        
    # Generate the UID
    if whoi_inst == True:
        uid = '-'.join(('CGINS',inst,sn))
        
    return uid

In [16]:
metadata_review['UID'] = metadata_review.apply(lambda x: generate_uid(x['CLASS-SERIES'], x['S/N']), axis=1)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


**====================================================================================================================**
Next, we need to classify the different metadata reviews based on the results with the following classes:
1. Good - this is the best result
2. Wrong calibration date - the date in the csv name differed from the calibration date and has been corrected to the calibration date of the instrument
3. Wrong calibration value - a calibration coefficient was identified as being incorrect, by either an exact match or by a comparative difference of > 0.001%
4. Missing - a csv is missing from asset management
5. Duplicate - a csv was identified as a duplicate or unneeded and removed

Individual csv files may fall into several categories. We

In [17]:
def wrong_cal_date(x):
    if type(x) == str:
        if 'no' in x.lower():
            return True
        else:
            return False
    else:
        return False

In [18]:
metadata_review['Wrong Date'] = metadata_review['Filename correct'].apply(wrong_cal_date)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
def wrong_cal_coef(x):
    if type(x) == str:
        if 'yes' in x.lower():
            return False
        else:
            return True
    elif np.isnan(x):
        return False
    else:
        return False

In [20]:
metadata_review['Wrong cal'] = metadata_review['Cal coeff match'].apply(wrong_cal_coef)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
def is_missing(x):
    if type(x) is str:
        if x.lower() == 'new':
            return True
        else:
            return False
    else:
        return False

In [22]:
metadata_review['Is missing'] = metadata_review['Duplicate'].apply(is_missing)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
def is_duplicate(x):
    if type(x) is str:
        if x.lower() == 'yes':
            return True
        else:
            return False
    else:
        return False        

In [24]:
metadata_review['Is duplicate'] = metadata_review['Duplicate'].apply(is_duplicate)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Now that I have functions to test for four of the five situations, I can check against the fifth situation to determine if the file is "good," i.e. it didn't fail any of the other tests.

In [25]:
def is_good(x):
    if any(x) == True:
        return False
    else:
        return True
    

In [26]:
metadata_review['Is good'] = metadata_review[['Wrong Date', 'Wrong cal', 'Is missing', 'Is duplicate']].apply(is_good, axis=1)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
metadata_review[['Wrong Date','Wrong cal','Is missing','Is duplicate','Is good']].head(10)

,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,True,False,False,False,False
4,False,False,False,False,True
5,False,False,False,False,True
6,True,False,False,False,False
7,True,False,False,False,False
8,False,False,False,False,True
9,False,False,False,False,True


**====================================================================================================================**
Next we want to construct the new CSV filenames based on the calibration date corrections. These are the names which will show up in the OOINet system. We can build the new CSV filenames from the instrument UID, which was previously bilt from the instrument class-series and serial number, and the correct/corrected calibration date.


In [28]:
def convert_df_time(x):
    if type(x) == datetime.datetime:
        return x.strftime('%Y%m%d')
    elif type(x) == int:
        return pd.to_datetime(x).strftime('%Y%m%d')
    else:
        pass

In [29]:
metadata_review['Cal Date'] = metadata_review['Cal Date'].apply(convert_df_time)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
def new_csv_filename(x):
    og_csv = x['Original Calibration CSV']
    if not og_csv.endswith('.csv') and og_csv != None:
        og_csv = og_csv + '.csv'
        x['Original Calibration CSV'] = og_csv
        
    if x['Is duplicate']:
        return np.nan
    elif x['Cal Date'] == None:
        return og_csv
    elif x['Wrong Date'] or x['Is missing']:
        new_csv = x['UID'] + '__' + x['Cal Date'] + '.csv'
        if new_csv == x['Original Calibration CSV'] and not x['Is missing']:
            print("Check calibration date for {} for errors.".format(x['Original Calibration CSV']))
        else:
            return new_csv
    else:
        return x['Original Calibration CSV']

In [31]:
metadata_review['New Calibration CSV'] = metadata_review.apply(new_csv_filename, axis=1)

Check calibration date for CGINS-ADCPSL-24661__20170626.csv for errors.
Check calibration date for CGINS-ADCPSL-24664__20170626.csv for errors.
Check calibration date for CGINS-ADCPSJ-22642__20170405.csv for errors.
Check calibration date for CGINS-ADCPSN-23579__20170109.csv for errors.


/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
metadata_review[metadata_review['New Calibration CSV'] == 'None.csv']

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,...,Pull request verified primary,Pull request verified secondary,Unnamed: 14,UID,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good,New Calibration CSV


In [33]:
set(metadata_review['CLASS-SERIES'])

{'ADCPS-J',
 'ADCPS-L',
 'ADCPS-N',
 'ADCPT-F',
 'ADCPT-G',
 'CTDBP-C',
 'CTDBP-D',
 'CTDBP-E',
 'CTDBP-F',
 'CTDBP-P',
 'DOSTA-D',
 'FLORD-G',
 'FLORT-D',
 'METBKA',
 'NUTNR-B',
 'OPTAAD',
 'PCO2W-B',
 'PCO2W-C',
 'PHSEN-D',
 'PHSEN-E',
 'PHSEN-F',
 'PRESF-B',
 'PRESF-C',
 'SPKIRB'}

**====================================================================================================================**
### Demonstration: SPKIRBs
This section is to work through the approach to generating the requisite information need to fill out the UW metadata communication spreadsheet based upon CGSN's metadata review approach. I need to gather the following information for the spreadsheet:
* Array
* Platform
* Node
* Instrument
* RefDes
* Asset UID -  
* Serial
* Deployment(s)
* Github Change Date - I don't think this is necessary, since it doesn't affect the end user until a release and ingestion to OOINet
* OOI Change Date - Question on 
* CSV file name - this is the filename which is in the system (so changes which have not been pushed to OOI net should not be put on the spreadsheet?)
* Github URL - Is this also necessary, when they can directly call (via M2M) or download the calibration information from the Portal
* Change type - This is my 5 categories from above
* dateRange Start
* dateRange End
* Annotation

Starting with my Metadata tracking spreadsheet above, I want to be able to use a series of M2M calls to the OOI API in order to get the data necessary to fill out the spreadsheet above. To test the possible approaches, I'm going to use the SPKIR-B instrument series, whose calibration changes have been pushed to OOI-Integration and Released into Production. 

In [34]:
spkirb = metadata_review[metadata_review['CLASS-SERIES'] == 'SPKIRB']
spkirb

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,...,Pull request verified primary,Pull request verified secondary,Unnamed: 14,UID,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good,New Calibration CSV
1135,SPKIRB,229,20121113,CGINS-SPKIRB-00229__20150315.csv,Yes,CC_immersion_factor,"No, need date changed",AR; updated filename in Vault RT\nUpdated CSV ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,True,False,False,False,CGINS-SPKIRB-00229__20121113.csv
1136,SPKIRB,229,20160316,CGINS-SPKIRB-00229__20161027.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,False,False,False,False,CGINS-SPKIRB-00229__20160316.csv
1137,SPKIRB,229,20180420,CGINS-SPKIRB-00229__20180420.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,NaN,NaN,NaN,CGINS-SPKIRB-00229,False,False,False,False,True,CGINS-SPKIRB-00229__20180420.csv
1138,SPKIRB,230,20121113,CGINS-SPKIRB-00230__20150315.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20121113.csv
1139,SPKIRB,230,20160316,CGINS-SPKIRB-00230__20161027.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20160316.csv
1140,SPKIRB,230,20180420,CGINS-SPKIRB-00230__20180418.csv,Yes,Yes,"No, need date changed",AR,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20180420.csv
1141,SPKIRB,238,20130829,CGINS-SPKIRB-00238__20131121.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: Calibration Files > DI7238A.cal,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20130829.csv
1142,SPKIRB,238,20150407,CGINS-SPKIRB-00238__20151114.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20150407.csv
1143,SPKIRB,238,20170911,CGINS-SPKIRB-00238__20170913.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20170911.csv
1144,SPKIRB,238,20180601,CGINS-SPKIRB-00238__20180601.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,NaN,NaN,NaN,CGINS-SPKIRB-00238,False,False,False,False,True,CGINS-SPKIRB-00238__20180601.csv


Get the first instance of the spkir-b:

In [35]:
spkirb[spkirb['UID'] == 'CGINS-SPKIRB-00229']

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,...,Pull request verified primary,Pull request verified secondary,Unnamed: 14,UID,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good,New Calibration CSV
1135,SPKIRB,229,20121113,CGINS-SPKIRB-00229__20150315.csv,Yes,CC_immersion_factor,"No, need date changed",AR; updated filename in Vault RT\nUpdated CSV ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,True,False,False,False,CGINS-SPKIRB-00229__20121113.csv
1136,SPKIRB,229,20160316,CGINS-SPKIRB-00229__20161027.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,False,False,False,False,CGINS-SPKIRB-00229__20160316.csv
1137,SPKIRB,229,20180420,CGINS-SPKIRB-00229__20180420.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,NaN,NaN,NaN,CGINS-SPKIRB-00229,False,False,False,False,True,CGINS-SPKIRB-00229__20180420.csv


**====================================================================================================================** 
### Deployment Information
Try querying the deployment information for a given instrument UID.

In [36]:
url = '/'.join((base_url,'12587','asset','deployments','CGINS-SPKIRB-00229?editphase=ALL'))
url

'https://ooinet.oceanobservatories.org/api/m2m/12587/asset/deployments/CGINS-SPKIRB-00229?editphase=ALL'

In [37]:
data = requests.get(url, auth=(username, token)).json()

In [38]:
deploydf = pd.DataFrame(data)
deploydf.sort_values(by='deploymentNumber', inplace=True)
deploydf.reset_index(drop=True, inplace=True)
deploydf

,deployCruiseIdentifier,deploymentNumber,depth,editPhase,endTime,eventId,latitude,longitude,mooring_uid,node,node_uid,orbitRadius,recoverCruiseIdentifier,sensor,sensor_uid,startTime,subsite,versionNumber,waterDepth
0,AT-26-30,1,-5.0,OPERATIONAL,1.448532e+12,3113,-42.92283,-42.455830,CGMGA-01SUMO-00001,SBD11,None,None,NBP-15-10,05-SPKIRB000,CGINS-SPKIRB-00229,1426454700000,GA01SUMO,1,5181.0
1,NBP16-09,3,0.0,OPERATIONAL,1.515925e+12,3210,-42.92040,-42.440932,CGMGA-01SUMO-00003,SBD11,None,None,AT39-03,05-SPKIRB000,CGINS-SPKIRB-00229,1477532820000,GA01SUMO,1,5175.0
2,AR-34,10,7.0,OPERATIONAL,NaN,4487,40.36723,-70.881470,CGMCP-03ISSM-00010,RID26,None,None,None,08-SPKIRB000,CGINS-SPKIRB-00229,1554648120000,CP03ISSM,1,93.0


In [39]:
deploydf.columns

Index(['deployCruiseIdentifier', 'deploymentNumber', 'depth', 'editPhase',
       'endTime', 'eventId', 'latitude', 'longitude', 'mooring_uid', 'node',
       'node_uid', 'orbitRadius', 'recoverCruiseIdentifier', 'sensor',
       'sensor_uid', 'startTime', 'subsite', 'versionNumber', 'waterDepth'],
      dtype='object')

Okay, querying the Deployment information for the instrument UID gives me the following key information:
* Deployment Number
* Node
* Subsite
* Sensor
* Start Time
* End Time

However, we are lacking the following information:
* Instrument
* Reference Designator
* CSV File

Thus, we still cannot directly link a particular CSV file to a given deployment yet. Now, we want to try to query the calibration information associated with a particular deployment for the given instrument:

In [40]:
subsite = deploydf['subsite']
node = deploydf['node']
sensor = deploydf['sensor']
depnum = deploydf['deploymentNumber']
startTime = deploydf['startTime']
endTime = deploydf['endTime']

In [41]:
url = '/'.join((base_url,'12587','events','deployment','inv',subsite[0],node[0],sensor[0],str(depnum[0])))
url

'https://ooinet.oceanobservatories.org/api/m2m/12587/events/deployment/inv/GA01SUMO/SBD11/05-SPKIRB000/1'

In [42]:
data = requests.get(url, auth=(username, token)).json()

In [43]:
data[0].keys()

dict_keys(['@class', 'mooring', 'editPhase', 'node', 'deploymentNumber', 'versionNumber', 'referenceDesignator', 'sensor', 'deployCruiseInfo', 'recoverCruiseInfo', 'deployedBy', 'recoveredBy', 'inductiveId', 'waterDepth', 'ingestInfo', 'location', 'assetUid', 'eventId', 'eventType', 'eventName', 'eventStartTime', 'eventStopTime', 'notes', 'tense', 'dataSource', 'lastModifiedTimestamp'])

In [44]:
data[0]['sensor'].keys()

dict_keys(['@class', 'calibration', 'events', 'remoteResources', 'assetId', 'serialNumber', 'manufacturer', 'physicalInfo', 'editPhase', 'description', 'notes', 'ooiPartNumber', 'ooiSerialNumber', 'deliveryOrderNumber', 'institutionPropertyNumber', 'institutionPurchaseOrderNumber', 'shelfLifeExpirationDate', 'firmwareVersion', 'softwareVersion', 'powerRequirements', 'assetType', 'mobile', 'modelNumber', 'purchasePrice', 'purchaseDate', 'deliveryDate', 'depthRating', 'ooiPropertyNumber', 'uid', 'name', 'location', 'owner', 'dataSource', 'lastModifiedTimestamp'])

In [45]:
data[0]['sensor']['calibration'][0]

{'@class': '.XCalibration',
 'calData': [{'@class': '.XCalibrationData',
   'comments': None,
   'value': [2.1951801819e-07,
    2.13132061419e-07,
    2.0729377062e-07,
    2.1894280634e-07,
    2.24599279252e-07,
    2.21514022543e-07,
    2.32288794212e-07],
   'assetUid': 'CGINS-SPKIRB-00229',
   'eventId': 25726,
   'eventType': 'CALIBRATION_DATA',
   'eventName': 'CC_scale',
   'eventStartTime': 1458086400000,
   'eventStopTime': None,
   'notes': None,
   'tense': 'UNKNOWN',
   'dataSource': 'CGINS-SPKIRB-00229__20160316_Cal_Info.xlsx',
   'lastModifiedTimestamp': 1565831106432},
  {'@class': '.XCalibrationData',
   'comments': None,
   'value': [2.0494027232e-07,
    1.99166952052e-07,
    1.95802011457e-07,
    2.11719179789e-07,
    2.19613129523e-07,
    2.12561891831e-07,
    2.22801253463e-07],
   'assetUid': 'CGINS-SPKIRB-00229',
   'eventId': 45739,
   'eventType': 'CALIBRATION_DATA',
   'eventName': 'CC_scale',
   'eventStartTime': 1524182400000,
   'eventStopTime': Non

So, querying the calibration data for a specific UID for a specific deployment number does not, in fact, return just that deployment information. It returns the _all_ of the calibration info, separated based on the calibration coefficient name. This is a problem, because I want _only_ the calibration info for a specific deployment without having to dig into each and every calibration coefficient checking for the closest startTime and endTime.

We're going to double check the deployment info by just querying for the deployment numbers for a specified instrument:

In [46]:
url = '/'.join((base_url,'12587','events','deployment','inv',subsite[0],node[0],sensor[0]))
url

'https://ooinet.oceanobservatories.org/api/m2m/12587/events/deployment/inv/GA01SUMO/SBD11/05-SPKIRB000'

In [47]:
data = requests.get(url, auth=(username, token)).json()
data

[1, 2, 3]

Okay, now I am confused. Requesting this specific deployment data returns [1,2,3]. This is probably because a new deployment starts before an old deployment stops, returning deployment 2. Additionally, this only returns the time that the instrument was located on that specific array, site, and node. It does not return all the calibration coefficients for a specific UID.

In [48]:
startTime.apply(convert_ooi_time)

0   2015-03-15 21:25:00
1   2016-10-27 01:47:00
2   2019-04-07 14:42:00
Name: startTime, dtype: datetime64[ns]

In [49]:
# Add one day to the start time to limit to a single deployment
dt = 1*24*60*60*1000
dt

86400000

In [50]:
T1 = convert_ooi_time(startTime[0]).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
T2 = convert_ooi_time(startTime[0]+dt).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
(T1, T2)

('2015-03-15T21:25:00.000000Z', '2015-03-16T21:25:00.000000Z')

In [51]:
url = '/'.join((base_url,'12587','asset','cal?uid=CGINS-SPKIRB-00229&beginDT={}&endDT={}'.format(T1,T2)))
url

'https://ooinet.oceanobservatories.org/api/m2m/12587/asset/cal?uid=CGINS-SPKIRB-00229&beginDT=2015-03-15T21:25:00.000000Z&endDT=2015-03-16T21:25:00.000000Z'

In [52]:
data = requests.get(url, auth=(username, token)).json()

In [53]:
data

{'@class': '.XInstrument',
 'calibration': [{'@class': '.XCalibration',
   'calData': [{'@class': '.XCalibrationData',
     'comments': None,
     'value': [2.02432408562e-07,
      2.00522864189e-07,
      1.93073886022e-07,
      2.04773838156e-07,
      2.17142782368e-07,
      2.1046694894e-07,
      2.17434944825e-07],
     'assetUid': 'CGINS-SPKIRB-00229',
     'eventId': 53758,
     'eventType': 'CALIBRATION_DATA',
     'eventName': 'CC_scale',
     'eventStartTime': 1352764800000,
     'eventStopTime': None,
     'notes': None,
     'tense': 'UNKNOWN',
     'dataSource': 'CGINS-SPKIRB-00229__20121113_Cal_Info.xlsx',
     'lastModifiedTimestamp': 1565831193976}],
   'name': 'CC_scale'},
  {'@class': '.XCalibration',
   'calData': [{'@class': '.XCalibrationData',
     'comments': None,
     'value': [1.368, 1.41, 1.365, 1.354, 1.372, 1.322, 1.347],
     'assetUid': 'CGINS-SPKIRB-00229',
     'eventId': 53759,
     'eventType': 'CALIBRATION_DATA',
     'eventName': 'CC_immersion_f

This provides me with the calibration information for a single deployment based on the instrument UID rather than having to construct the full reference designator, which is what I want. Now, I need to loop through all of the deployments and return the "data source" along with the "last modified time stamp"

In [54]:
dataSource = ()
lastModifiedTimestamp = ()
instrument = ()
serialNum = ()
for i in startTime:
    T1 = convert_ooi_time(i).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    T2 = convert_ooi_time(i+dt).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    url = '/'.join((base_url,'12587','asset','cal?uid=CGINS-SPKIRB-00229&beginDT={}&endDT={}'.format(T1,T2)))
    calData = requests.get(url, auth=(username, token)).json()
    inst = calData['description']
    instrument = instrument + (inst,)
    sn = calData['serialNumber']
    serialNum = serialNum + (sn,)
    Source = calData['calibration'][0]['calData'][0]['dataSource']
    dataSource = dataSource + (Source,)
    lastMod = calData['calibration'][0]['calData'][0]['lastModifiedTimestamp']
    lastModifiedTimestamp = lastModifiedTimestamp + (lastMod,)
deploydf['dataSource'] = dataSource
deploydf['lastModifiedTimestamp'] = lastModifiedTimestamp
deploydf['instrument'] = instrument
deploydf['serialNum'] = serialNum

In [55]:
deploydf.columns

Index(['deployCruiseIdentifier', 'deploymentNumber', 'depth', 'editPhase',
       'endTime', 'eventId', 'latitude', 'longitude', 'mooring_uid', 'node',
       'node_uid', 'orbitRadius', 'recoverCruiseIdentifier', 'sensor',
       'sensor_uid', 'startTime', 'subsite', 'versionNumber', 'waterDepth',
       'dataSource', 'lastModifiedTimestamp', 'instrument', 'serialNum'],
      dtype='object')

**====================================================================================================================**
Now, I want to construct a dataframe which contains the relevant information to fill out the Metadata Changes & Communications spreadsheet. Start by building up a dictionary, then put it into a dataframe.

In [56]:
def reformat_dataSource(x):
    new = x.replace('_Cal_Info.xlsx','.csv')
    return new

In [57]:
reformat_dataSource(deploydf['dataSource'].iloc[0])

'CGINS-SPKIRB-00229__20121113.csv'

In [58]:
deploydf['dataSource'] = deploydf['dataSource'].apply(reformat_dataSource)
deploydf['dataSource']

0    CGINS-SPKIRB-00229__20121113.csv
1    CGINS-SPKIRB-00229__20160316.csv
2    CGINS-SPKIRB-00229__20180420.csv
Name: dataSource, dtype: object

In [59]:
deploydf['RefDes'] = deploydf['subsite'] + '-' + deploydf['node'] + '-' + deploydf['sensor']
deploydf['RefDes']

0    GA01SUMO-SBD11-05-SPKIRB000
1    GA01SUMO-SBD11-05-SPKIRB000
2    CP03ISSM-RID26-08-SPKIRB000
Name: RefDes, dtype: object

Create a subselection of columns from the deploy dataframe which contains the relevant data 

In [60]:
calData = deploydf[['subsite','node','instrument','RefDes','sensor_uid','deploymentNumber','lastModifiedTimestamp','dataSource','startTime','endTime']]

In [61]:
calData

,subsite,node,instrument,RefDes,sensor_uid,deploymentNumber,lastModifiedTimestamp,dataSource,startTime,endTime
0,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,1,1565831193976,CGINS-SPKIRB-00229__20121113.csv,1426454700000,1.448532e+12
1,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,3,1565831106432,CGINS-SPKIRB-00229__20160316.csv,1477532820000,1.515925e+12
2,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,10,1565831156123,CGINS-SPKIRB-00229__20180420.csv,1554648120000,NaN


Now, I can merge the subselected dataframe above with the metadata review based on the key of 'New Calibration CSV'::'dataSource'. This should provide us with all the necessary data to fill the spreadsheet (that can filled out from M2M calls)

In [62]:
spkirb229 = spkirb[spkirb['UID'] == 'CGINS-SPKIRB-00229']

In [63]:
spkirb229 = spkirb229.merge(deploydf, left_on='New Calibration CSV', right_on='dataSource')


In [64]:
spkirb229.columns

Index(['CLASS-SERIES', 'S/N', 'Cal Date', 'Original Calibration CSV',
       'Vendor Docs exist', 'Cal coeff match', 'Filename correct',
       'In progress?', 'Duplicate', 'Notes', 'Pull request #',
       'Date pull request submitted', 'Pull request verified primary',
       'Pull request verified secondary', 'Unnamed: 14', 'UID', 'Wrong Date',
       'Wrong cal', 'Is missing', 'Is duplicate', 'Is good',
       'New Calibration CSV', 'deployCruiseIdentifier', 'deploymentNumber',
       'depth', 'editPhase', 'endTime', 'eventId', 'latitude', 'longitude',
       'mooring_uid', 'node', 'node_uid', 'orbitRadius',
       'recoverCruiseIdentifier', 'sensor', 'sensor_uid', 'startTime',
       'subsite', 'versionNumber', 'waterDepth', 'dataSource',
       'lastModifiedTimestamp', 'instrument', 'serialNum', 'RefDes'],
      dtype='object')

In [65]:
cols = ('Array','Platform','Node','Instrument','RefDes','Asset ID','Serial Number','deployment','gitHub changeDate',
        'OOI changeDate','file','URL','changeType','dateRangeStart','dateRangeEnd','annotation','Wrong Date',
       'Wrong cal','Is missing','Is duplicate','Is good')

In [66]:
name_map = {
    'Array':None,
    'Platform':'subsite',
    'Node':'node',
    'Instrument':'instrument',
    'RefDes':'RefDes',
    'Asset ID':'UID',
    'Serial Number':'serialNum',
    'deployment':'deploymentNumber',
    'gitHub changeDate':'Pull request #',
    'OOI changeDate':'lastModifiedTimestamp',
    'file':'dataSource',
    'dateRangeStart':'startTime',
    'dateRangeEnd':'endTime',
    'annotation':None,
    'Wrong Date':'Wrong Date',
    'Wrong cal':'Wrong cal',
    'Is missing':'Is missing',
    'Is duplicate':'Is duplicate',
    'Is good':'Is good'
}

In [67]:
comdf = pd.DataFrame(columns=cols)
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,...,URL,changeType,dateRangeStart,dateRangeEnd,annotation,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good


In [68]:
for i in cols:
    if name_map.get(i) is not None:
        comdf[i] = spkirb229[name_map.get(i)]

In [69]:
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,...,URL,changeType,dateRangeStart,dateRangeEnd,annotation,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good
0,NaN,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,1,39,1565831193976,...,NaN,NaN,1426454700000,1.448532e+12,NaN,True,True,False,False,False
1,NaN,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,3,39,1565831106432,...,NaN,NaN,1477532820000,1.515925e+12,NaN,True,False,False,False,False
2,NaN,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,229,10,NaN,1565831156123,...,NaN,NaN,1554648120000,NaN,NaN,False,False,False,False,True


In [70]:
def generate_arrayName(x):
    if 'GA' in x:
        arrayName = 'Global Argentine Basin'
    elif 'GI' in x:
        arrayName = 'Global Irminger Sea'
    elif 'GP' in x:
        arrayName = 'Global Station Papa'
    elif 'GS' in x:
        arrayName = 'Global Southern Ocean'
    elif 'CP' in x:
        arrayName = 'Coastal Pioneer'
    else:
        arrayName = np.nan
    return arrayName

In [71]:
comdf['Array'] = comdf['Platform'].apply(generate_arrayName)
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,...,URL,changeType,dateRangeStart,dateRangeEnd,annotation,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good
0,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,1,39,1565831193976,...,NaN,NaN,1426454700000,1.448532e+12,NaN,True,True,False,False,False
1,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,3,39,1565831106432,...,NaN,NaN,1477532820000,1.515925e+12,NaN,True,False,False,False,False
2,Coastal Pioneer,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,229,10,NaN,1565831156123,...,NaN,NaN,1554648120000,NaN,NaN,False,False,False,False,True


In [72]:
comdf['OOI changeDate'] = comdf['OOI changeDate'].apply(convert_ooi_time)
comdf['dateRangeStart'] = comdf['dateRangeStart'].apply(convert_ooi_time)
comdf['dateRangeEnd'] = comdf['dateRangeEnd'].apply(convert_ooi_time)

In [73]:
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,...,URL,changeType,dateRangeStart,dateRangeEnd,annotation,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good
0,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,1,39,2019-08-15 01:06:33.976,...,NaN,NaN,2015-03-15 21:25:00,2015-11-26 10:08:00,NaN,True,True,False,False,False
1,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,3,39,2019-08-15 01:05:06.432,...,NaN,NaN,2016-10-27 01:47:00,2018-01-14 10:24:00,NaN,True,False,False,False,False
2,Coastal Pioneer,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,229,10,NaN,2019-08-15 01:05:56.123,...,NaN,NaN,2019-04-07 14:42:00,NaT,NaN,False,False,False,False,True


In [74]:
def generate_gitHub_url(x):
    base_url = 'https://github.com/ooi-integration/asset-management/blob/master/calibration'
    inst = x.split('-')[1]
    full_url = '/'.join((base_url,inst,x))
    return full_url

In [75]:
comdf['URL'] = comdf['file'].apply(generate_gitHub_url)
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,...,URL,changeType,dateRangeStart,dateRangeEnd,annotation,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good
0,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,1,39,2019-08-15 01:06:33.976,...,https://github.com/ooi-integration/asset-manag...,NaN,2015-03-15 21:25:00,2015-11-26 10:08:00,NaN,True,True,False,False,False
1,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,3,39,2019-08-15 01:05:06.432,...,https://github.com/ooi-integration/asset-manag...,NaN,2016-10-27 01:47:00,2018-01-14 10:24:00,NaN,True,False,False,False,False
2,Coastal Pioneer,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,229,10,NaN,2019-08-15 01:05:56.123,...,https://github.com/ooi-integration/asset-manag...,NaN,2019-04-07 14:42:00,NaT,NaN,False,False,False,False,True


In [76]:
def classify_changeType(x):
    statement = ''
    if x['Is good'] == True:
        return 'No errors found'
    elif x['Is missing'] == True:
        return 'Missing file added'
    elif x['Is duplicate'] == True:
        return 'File deleted'
    else:
        if x['Wrong Date'] == True:
            statement = statement + 'File renamed with correct date '
        if x['Wrong cal'] == True:
            statement = statement + 'Calibration coefficients were modified'
        return statement

In [77]:
comdf['changeType'] = comdf[['Wrong Date','Wrong cal','Is missing','Is duplicate','Is good']].apply(classify_changeType, axis=1)

In [78]:
comdf.drop(columns=['Wrong Date','Wrong cal','Is missing','Is duplicate','Is good'], inplace=True)
comdf

,Array,Platform,Node,Instrument,RefDes,Asset ID,Serial Number,deployment,gitHub changeDate,OOI changeDate,file,URL,changeType,dateRangeStart,dateRangeEnd,annotation
0,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,1,39,2019-08-15 01:06:33.976,CGINS-SPKIRB-00229__20121113.csv,https://github.com/ooi-integration/asset-manag...,File renamed with correct date Calibration coe...,2015-03-15 21:25:00,2015-11-26 10:08:00,NaN
1,Global Argentine Basin,GA01SUMO,SBD11,Spectral Irradiance: SPKIR Series B,GA01SUMO-SBD11-05-SPKIRB000,CGINS-SPKIRB-00229,229,3,39,2019-08-15 01:05:06.432,CGINS-SPKIRB-00229__20160316.csv,https://github.com/ooi-integration/asset-manag...,File renamed with correct date,2016-10-27 01:47:00,2018-01-14 10:24:00,NaN
2,Coastal Pioneer,CP03ISSM,RID26,Spectral Irradiance: SPKIR Series B,CP03ISSM-RID26-08-SPKIRB000,CGINS-SPKIRB-00229,229,10,NaN,2019-08-15 01:05:56.123,CGINS-SPKIRB-00229__20180420.csv,https://github.com/ooi-integration/asset-manag...,No errors found,2019-04-07 14:42:00,NaT,NaN


**====================================================================================================================**
Now, I have successfully generated the spreadsheet with only a few wrinkles:
1. gitHub changeDate: I have put in the Pull Request (PR) number here instead of a date. I can hopefully match those numbers up at a later date by identifying when the PR was merged with master and pushed to ooi-integration
2. annotation: I have not populated the annotation format yet, and part of the reason is:
3. downstream: I have not determined downstream impacts for a given file change. This can be added later.

Next, I want to structure this into a scriptable, automated format that makes use of some key identifiers that avoid doing this on a instrument-by-instrument basis

Start by selecting an instrument class-series, preferably one which where the review has been finished and pushed to ooi-integration.

In [84]:
metadata_review['CLASS-SERIES'] = metadata_review['CLASS-SERIES'].apply(lambda x: x.replace('-',''))

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
instrument = 'SPKIRB'

Select from the metadata review the relevant instruments:

In [86]:
np.unique(metadata_review['CLASS-SERIES'])

array(['ADCPSJ', 'ADCPSL', 'ADCPSN', 'ADCPTF', 'ADCPTG', 'CTDBPC',
       'CTDBPD', 'CTDBPE', 'CTDBPF', 'CTDBPP', 'DOSTAD', 'FLORDG',
       'FLORTD', 'METBKA', 'NUTNRB', 'OPTAAD', 'PCO2WB', 'PCO2WC',
       'PHSEND', 'PHSENE', 'PHSENF', 'PRESFB', 'PRESFC', 'SPKIRB'],
      dtype=object)

In [87]:
meta = metadata_review[metadata_review['CLASS-SERIES'] == instrument]
meta

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,...,Pull request verified primary,Pull request verified secondary,Unnamed: 14,UID,Wrong Date,Wrong cal,Is missing,Is duplicate,Is good,New Calibration CSV
1135,SPKIRB,229,20121113,CGINS-SPKIRB-00229__20150315.csv,Yes,CC_immersion_factor,"No, need date changed",AR; updated filename in Vault RT\nUpdated CSV ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,True,False,False,False,CGINS-SPKIRB-00229__20121113.csv
1136,SPKIRB,229,20160316,CGINS-SPKIRB-00229__20161027.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00229,True,False,False,False,False,CGINS-SPKIRB-00229__20160316.csv
1137,SPKIRB,229,20180420,CGINS-SPKIRB-00229__20180420.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_229_Calibration_...,...,NaN,NaN,NaN,CGINS-SPKIRB-00229,False,False,False,False,True,CGINS-SPKIRB-00229__20180420.csv
1138,SPKIRB,230,20121113,CGINS-SPKIRB-00230__20150315.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20121113.csv
1139,SPKIRB,230,20160316,CGINS-SPKIRB-00230__20161027.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20160316.csv
1140,SPKIRB,230,20180420,CGINS-SPKIRB-00230__20180418.csv,Yes,Yes,"No, need date changed",AR,No,Source file: SPKIR_OCR-507_SN_230_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00230,True,False,False,False,False,CGINS-SPKIRB-00230__20180420.csv
1141,SPKIRB,238,20130829,CGINS-SPKIRB-00238__20131121.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: Calibration Files > DI7238A.cal,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20130829.csv
1142,SPKIRB,238,20150407,CGINS-SPKIRB-00238__20151114.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20150407.csv
1143,SPKIRB,238,20170911,CGINS-SPKIRB-00238__20170913.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,AR,SNW,NaN,CGINS-SPKIRB-00238,True,False,False,False,False,CGINS-SPKIRB-00238__20170911.csv
1144,SPKIRB,238,20180601,CGINS-SPKIRB-00238__20180601.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_238_Calibration_...,...,NaN,NaN,NaN,CGINS-SPKIRB-00238,False,False,False,False,True,CGINS-SPKIRB-00238__20180601.csv


In [88]:
uids = np.unique(meta['UID'])
uids

array(['CGINS-SPKIRB-00229', 'CGINS-SPKIRB-00230', 'CGINS-SPKIRB-00238',
       'CGINS-SPKIRB-00239', 'CGINS-SPKIRB-00240', 'CGINS-SPKIRB-00269',
       'CGINS-SPKIRB-00270', 'CGINS-SPKIRB-00271', 'CGINS-SPKIRB-00272',
       'CGINS-SPKIRB-00273', 'CGINS-SPKIRB-00274', 'CGINS-SPKIRB-00275',
       'CGINS-SPKIRB-00276', 'CGINS-SPKIRB-00288', 'CGINS-SPKIRB-00289',
       'CGINS-SPKIRB-00290', 'CGINS-SPKIRB-00291', 'CGINS-SPKIRB-00299',
       'CGINS-SPKIRB-00300', 'CGINS-SPKIRB-00301', 'CGINS-SPKIRB-00302'],
      dtype=object)

In [123]:
uid = uids[3]
uid

'CGINS-SPKIRB-00239'

In [124]:
udf = meta[meta['UID'] == uid]

Get the relevant deployment info:

In [125]:
def get_deployData(uid):
    url = '/'.join((base_url,'12587','asset','deployments',uid+'?editphase=ALL'))
    data = requests.get(url, auth=(username, token)).json()
    df = pd.DataFrame(data)
    df.sort_values(by='deploymentNumber', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [126]:
deploydf = get_deployData(uid)
deploydf

,deployCruiseIdentifier,deploymentNumber,depth,editPhase,endTime,eventId,latitude,longitude,mooring_uid,node,node_uid,orbitRadius,recoverCruiseIdentifier,sensor,sensor_uid,startTime,subsite,versionNumber,waterDepth
0,KN-221-4,1,-5.0,OPERATIONAL,1440241200000,5082,59.93370,-39.47378,CGMGI-01SUMO-00001,SBD11,None,None,AT-30-01,05-SPKIRB000,CGINS-SPKIRB-00239,1410375000000,GI01SUMO,1,2685.0
1,AR-07-01,3,-5.0,OPERATIONAL,1502703180000,5182,59.93410,-39.46730,CGMGI-01SUMO-00003,SBD11,None,None,AR21,05-SPKIRB000,CGINS-SPKIRB-00239,1468172640000,GI01SUMO,1,2691.0
2,AR30-03,5,-5.0,OPERATIONAL,1565337840000,5281,59.93295,-39.46549,CGMGI-01SUMO-00005,SBD11,None,None,AR35-05,05-SPKIRB000,CGINS-SPKIRB-00239,1528478460000,GI01SUMO,1,2693.0


In [127]:
def get_calData(uid, deployData):
    """
    This function takes in the instrument uid and a dataframe of the
    deployment information for the uid, and loops through all of the
    instrument deployments to return the calibration data for the
    instrument for each individual deployment.
    """
    
    startTime = deployData['startTime']
    dt = 8.64E10 # microseconds in a day
    
    # Initialize tuples for non-mutable storage of data
    dataSource = ()
    lastModifiedTimestamp = ()
    instrument = ()
    serialNumber = ()
    
    # Loop over the deployment startTime and get the data
    for t in startTime:
        T1 = convert_ooi_time(t).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        T2 = convert_ooi_time(t+dt).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        # Generate the url and get the calibration data for a single deployment
        url = '/'.join((base_url,'12587','asset','cal?uid='+uid+'&beginDT={}&endDT={}'.format(T1,T2)))
        calData = requests.get(url, auth=(username, token)).json()
        # Fill out the data tuples
        instrument = instrument + (calData['description'],)
        serialNumber = serialNumber + (calData['serialNumber'],)
        dataSource = dataSource + (calData['calibration'][0]['calData'][0]['dataSource'],)
        lastModifiedTimestamp = lastModifiedTimestamp + (calData['calibration'][0]['calData'][0]['lastModifiedTimestamp'],)
        
    # Now, put the data tuples into the deploy data dataframe
    deployData['dataSource'] = dataSource
    deployData['lastModifiedTimestamp'] = lastModifiedTimestamp
    deployData['instrument'] = instrument
    deployData['serialNumber'] = serialNumber
    
    # Return the expanded deployment data
    return deployData

In [128]:
deploydf = get_calData(uid, deploydf)
deploydf

,deployCruiseIdentifier,deploymentNumber,depth,editPhase,endTime,eventId,latitude,longitude,mooring_uid,node,...,sensor,sensor_uid,startTime,subsite,versionNumber,waterDepth,dataSource,lastModifiedTimestamp,instrument,serialNumber
0,KN-221-4,1,-5.0,OPERATIONAL,1440241200000,5082,59.93370,-39.47378,CGMGI-01SUMO-00001,SBD11,...,05-SPKIRB000,CGINS-SPKIRB-00239,1410375000000,GI01SUMO,1,2685.0,CGINS-SPKIRB-00239__20160203_Cal_Info.xlsx,1565831125135,Spectral Irradiance: SPKIR Series B,239
1,AR-07-01,3,-5.0,OPERATIONAL,1502703180000,5182,59.93410,-39.46730,CGMGI-01SUMO-00003,SBD11,...,05-SPKIRB000,CGINS-SPKIRB-00239,1468172640000,GI01SUMO,1,2691.0,CGINS-SPKIRB-00239__20160203_Cal_Info.xlsx,1565831125135,Spectral Irradiance: SPKIR Series B,239
2,AR30-03,5,-5.0,OPERATIONAL,1565337840000,5281,59.93295,-39.46549,CGMGI-01SUMO-00005,SBD11,...,05-SPKIRB000,CGINS-SPKIRB-00239,1528478460000,GI01SUMO,1,2693.0,CGINS-SPKIRB-00239__20171116_Cal_Info.xlsx,1565831115573,Spectral Irradiance: SPKIR Series B,239


In [130]:
def reformat_dataSource(x):
    new = x.replace('_Cal_Info.xlsx','.csv')
    return new

In [131]:
deploydf['dataSource'] = deploydf['dataSource'].apply(reformat_dataSource)
deploydf['RefDes'] = deploydf['subsite'] + '-' + deploydf['node'] + '-' + deploydf['sensor']

In [132]:
udf = udf.merge(deploydf, left_on='New Calibration CSV', right_on='dataSource')

In [133]:
udf

,CLASS-SERIES,S/N,Cal Date,Original Calibration CSV,Vendor Docs exist,Cal coeff match,Filename correct,In progress?,Duplicate,Notes,...,sensor_uid,startTime,subsite,versionNumber,waterDepth,dataSource,lastModifiedTimestamp,instrument,serialNumber,RefDes
0,SPKIRB,239,20160203,CGINS-SPKIRB-00239__20160710.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_239_Calibration_...,...,CGINS-SPKIRB-00239,1410375000000,GI01SUMO,1,2685.0,CGINS-SPKIRB-00239__20160203.csv,1565831125135,Spectral Irradiance: SPKIR Series B,239,GI01SUMO-SBD11-05-SPKIRB000
1,SPKIRB,239,20160203,CGINS-SPKIRB-00239__20160710.csv,Yes,Yes,"No, need date changed",AR; updated filename in Vault RT\nupdated csv ...,No,Source file: SPKIR_OCR-507_SN_239_Calibration_...,...,CGINS-SPKIRB-00239,1468172640000,GI01SUMO,1,2691.0,CGINS-SPKIRB-00239__20160203.csv,1565831125135,Spectral Irradiance: SPKIR Series B,239,GI01SUMO-SBD11-05-SPKIRB000
2,SPKIRB,239,20171116,CGINS-SPKIRB-00239__20171116.csv,Yes,Yes,Yes,AR,No,Source file: SPKIR_OCR-507_SN_239_Calibration_...,...,CGINS-SPKIRB-00239,1528478460000,GI01SUMO,1,2693.0,CGINS-SPKIRB-00239__20171116.csv,1565831115573,Spectral Irradiance: SPKIR Series B,239,GI01SUMO-SBD11-05-SPKIRB000


In [ ]:
# Function to make an API request and print the results
def get_and_print_api(url):
    r = requests.get(url, auth=(username, token))
    data = r.json()
    for d in data:
        print(d)

Now to track down all of the streams with a particular instrument...or is that doing this the wrong way?

In [ ]:
get_and_print_api('/'.join((base_url,sensor_url,'CP01CNSM','RID27')))

In [ ]:
# Put together a walk function which will return all of the reference designators for a particular instrument class
inst = 'SPKIRB'
url = '/'.join((base_url, sensor_url))
r = requests.get(url, auth=(username, token))
arrays = r.json()
arrays = [a for a in arrays if a.startswith('CP') or a.startswith('G')]
for array in arrays:
    url = '/'.join((base_url, sensor_url, array))
    nodes = requests.get(url, auth=(username, token)).json()
    for node in nodes:
        url = '/'.join((base_url, sensor_url, array, node))
        sensors = requests.get(url, auth=(username, token)).json()
        sensors = [s for s in sensors if 'CTDBP' in s]
        for sensor in sensors:
            url = '/'.join((base_url, sensor_url, array, node, sensor, 'metadata'))
            metadata = requests.get(url, auth=(username, token)).json()
            print(array + '/' + node + '/' + sensor + ': ' + metadata)

In [ ]:
arrays

In [ ]:
node